In [1]:
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [8]:
# !pip install pandas
!pip install matplotlib

     |████████████████████████████████| 9.4 MB 6.3 MB/s eta 0:00:01     |█████▉                          | 1.7 MB 6.3 MB/s eta 0:00:02
     |████████████████████████████████| 300 kB 20.1 MB/s eta 0:00:01
     |████████████████████████████████| 965 kB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 19.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 3.5 MB/s eta 0:00:01


In [2]:
from datetime import date, datetime
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 

In [3]:
os.getcwdb()

b'/home/worker/project/jp/jp_spark'

In [4]:
JDBC = {
    'url':'jdbc:oracle:thin:@fulfill_high?TNS_ADMIN=/home/worker/project/db/fulfill_wallet'
    ,'props':{
        'user':'dw_fulfill'
       ,'password':'fulfillment123QWE!@#'
    }   
}


In [5]:
JDBC['props']

{'user': 'dw_fulfill', 'password': 'fulfillment123QWE!@#'}

In [6]:
sys_stock = spark.read.jdbc(url=JDBC['url'], table='SYSTEM_STOCK',properties=JDBC['props'])

In [7]:
products = spark.read.jdbc(url=JDBC['url'], table='PRODUCTS',properties=JDBC['props'])

In [8]:
sys_stock.columns

['SS_KEY',
 'ARTICLE_NUMBER',
 'SUBINVENTORY',
 'LOCATION',
 'QUANTITY',
 'IN_DATE',
 'EXPIRY_DATE',
 'CURRENCY',
 'LOT_COST',
 'LOT_COST_IN_USD',
 'STD_DAY']

In [9]:
today = '2023-02-01'

In [10]:
# df_sys = spark.createDataFrame(sys_group)
sys_stock=sys_stock.where(col('STD_DAY')==today)

In [11]:
sys_group = sys_stock.groupBy(['ARTICLE_NUMBER','SUBINVENTORY']).count().collect()

In [14]:
df_sys = spark.createDataFrame(sys_group)


ValueError: can not infer schema from empty dataset

In [11]:
join_df = df_sys.join(products, on='ARTICLE_NUMBER', how='left').collect()


In [42]:
df_fin = spark.createDataFrame(join_df)


In [43]:
df_fin.where(col('SUBINVENTORY')=='KR_SERV01').show()

+--------------+------------+-----+-----------------+--------------------+--------------------+--------------------+----------------+-------+
|ARTICLE_NUMBER|SUBINVENTORY|count|          COUNTRY|         PROD_CENTRE|          PROD_GROUP|         DESCRIPTION|PROD_STATUS_TYPE|BIN_CUR|
+--------------+------------+-----+-----------------+--------------------+--------------------+--------------------+----------------+-------+
|      28958961|   KR_SERV01|    3|GEAS_IN_LS_KR_KRS|16 BIOPROCESS SER...|923 Bioprocess Do...|TC gasket 4in/ID9...|       DstFGActv| 2C1805|
|      29003827|   KR_SERV01|    2|GEAS_IN_LS_KR_KRS|48 RESEARCH SCIEN...|075 Lab AKTA Red ...|        LED PCB U9-L|       DstFGActv| 2C1703|
|      29207950|   KR_SERV01|    1|GEAS_IN_LS_KR_KRS|82 BIACORE CONSUM...|914 Buffers and s...|BIAnormalizing so...|       DstFGActv|   냉장|
|      29213860|   KR_SERV01|    1|GEAS_IN_LS_KR_KRS|16 BIOPROCESS SER...|920 Bioprocess Do...|Nozzle tip with o...|       DstFGActv| 2C1705|
|      2

In [44]:
df_fin = df_fin.select("*").withColumn('STD_DAY',lit(today))

In [57]:
df_fin = df_fin.withColumnRenamed('count','QUANTITY')

In [58]:
df_fin.columns

['ARTICLE_NUMBER',
 'SUBINVENTORY',
 'QUANTITY',
 'COUNTRY',
 'PROD_CENTRE',
 'PROD_GROUP',
 'DESCRIPTION',
 'PROD_STATUS_TYPE',
 'BIN_CUR',
 'STD_DAY']